In [1]:
# Load the packages needed for this project
# create spark and sparkcontext objects
from pyspark.sql import SparkSession
import numpy as np

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

import pyspark
from pyspark.ml import feature, regression, Pipeline, classification, pipeline, evaluation
from pyspark.sql import functions as fn, Row
from pyspark import sql

import matplotlib.pyplot as plt
import pandas as pd

In [2]:
#Load the data
book_example = pd.read_parquet('D:/Desktop/Jupyter Notebook/ist 718/project/optiver-realized-volatility-prediction/book_train.parquet/stock_id=0')
trade_example =  pd.read_parquet('D:/Desktop/Jupyter Notebook/ist 718/project/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=0')
book_train = pd.read_csv('D:/Desktop/Jupyter Notebook/ist 718/project/optiver-realized-volatility-prediction/train.csv')
stock_id = '0'
book_example.loc[:,'stock_id'] = stock_id
trade_example.loc[:,'stock_id'] = stock_id
# Ground truth: Stock_id=0
booktrain_0= book_train[book_train['stock_id']==0]

In [3]:
# Extract distinct time_id (There are the same time windows in all datasets of 127 stocks)
book_spark_example= spark.createDataFrame(book_example)
book_spark_example.registerTempTable('book_spark_example')
time_id_index= spark.sql('select time_id from book_spark_example').dropDuplicates().\
                       select('time_id').toPandas().sort_values('time_id').reset_index(drop=True)     

In [4]:
# Splits
time_id_index_1=spark.createDataFrame(time_id_index)
training_df_index, validation_df_index, testing_df_index = time_id_index_1.randomSplit([0.6, 0.3, 0.1], seed=0)
training_df_index, validation_df_index, testing_df_index = training_df_index.toPandas(),\
                                                            validation_df_index.toPandas(), testing_df_index.toPandas()
training_df_index= np.array(training_df_index['time_id'])
validation_df_index=  np.array(validation_df_index['time_id'])
testing_df_index= np.array(testing_df_index['time_id'])
# Index match
def dataset_split(row_index):
    df=pd.DataFrame()
    for n in row_index:
        df= df.append(book_example[book_example['time_id']==n])
    return df
training_df= dataset_split(training_df_index)
validation_df = dataset_split(validation_df_index)
testing_df = dataset_split(testing_df_index)

In [5]:
def trade_split(row_index):
    df=pd.DataFrame()
    for n in row_index:
        df= df.append(trade_example[trade_example['time_id']==n])
    return df
training_trade_df= trade_split(training_df_index)
validation_trade_df = trade_split(validation_df_index)
testing_trade_df = trade_split(testing_df_index)

In [6]:
# Book splits 
#training_df
#validation_df
#testing_df

# Trade splits
#training_trade_df
#validation_trade_df
#testing_trade_df